In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()
plt.rcParams.update({'figure.facecolor':'white'})

# Load data and do basic formatting

In [ ]:
df = pd.read_csv("../data/processed_sensor_dwd_train.csv", index_col=0)

In [ ]:
# convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# convert pressure to hPa
df['pressure_sensors'] = df['pressure_sensors'] / 100
df['pressure_std'] = df['pressure_std'] / 100

# add sensor IDs
df_location = df.groupby(['lat', 'lon']).count().reset_index()[['lat', 'lon']]
df_location['location_id'] = df_location.index+1
df = df.merge(df_location, on=['lat', 'lon'], how='left')

# define lists with columns
no_data_cols = ['location_id', 'timestamp', 'city', 'lat', 'lon']
sc_cols = sorted(['PM10', 'PM2p5', 'PM10_std', 'PM2p5_std', 'pressure_sensors', 'temperature_sensors', 'humidity_sensors', 'pressure_std', 'temperature_std', 'humidity_std'])
sc_cols_wo_std = [col for col in sc_cols if 'std' not in col]
dwd_cols = sorted([col for col in df.columns if (col not in no_data_cols and col not in sc_cols)])
std_cols = [col for col in sc_cols if 'std' in col]
data_cols_wo_std = sc_cols_wo_std + dwd_cols
data_cols = sc_cols + dwd_cols

# reorganize columns: first non-data columns, then sorted data columns
df = df.reindex(columns=no_data_cols + sc_cols + dwd_cols)
df

In [ ]:
# save assignment of sensor_id to coordinates
location_id_assignment = pd.DataFrame(
    data={
        'location_id': df['location_id'].unique()
    }
)
for l in ['lat', 'lon']:
    location_id_assignment[l] = location_id_assignment.apply(lambda x: df.loc[df['location_id']==x['location_id'], l].iloc[0], axis=1)


In [ ]:
df.info()

# Investigation of missing values, zeros and outliers

In [ ]:
# Basic statistics of the whole sc dataset
df[sc_cols_wo_std].describe().T.round(1)

In [ ]:
# Basic statistics of the sc dataset for Frankfurt
df[df['city']=='Frankfurt'][sc_cols_wo_std].describe().T.round(1)

In [ ]:
# Basic statistics of the sc dataset for Bremen
df[df['city']=='Bremen'][sc_cols_wo_std].describe().T.round(1)

PM10: Mean is almost double of the 75th percentile -> Outliers raise the mean extremely </br>
PM2.5: similar to PM10, but less extreme </br>
humidity: al values (mean, 25th, 50th and 75th percentile) seem to be very large, the max value is above 100, what doesn't make any sense </br>
pressure: assuming the units are Pa (1 bar = 100.000 Pa): min value is below 100 -> unrealistic, max value is also unrealistic (more than 60 bar) </br>
temperature: std seems very high (54 °C), min and max value are unrealistic </br>
 </br>
 Bremen vs. Frankfurt </br>
 PM10 and PM2.5: std for Bremen is double of std for Frankfurt </br>
 humidity: 50th percentile of Bremen is already 99.9 % what seems quite high
 pressure and temperature: no obvious unrealistic observations besides the min and max values

In [ ]:
print("missing values in each column")
for col in df.columns:
    print(f"{col}: {df[col].isna().sum()} ({round(df[col].isna().sum() / df.shape[0] * 100, 1)} %)")

In [ ]:
print("value '0' in each column")
for col in df.columns:
    print(f"{col}: {df[df[col]==0][col].count()} ({round(df[df[col]==0][col].count() / df.shape[0] * 100, 1)} %)")

In [ ]:
def count_nan_and_0s(df: pd.DataFrame, cols: list = None) -> pd.DataFrame:
    """Counts zeros and nans per column.

    Args:
        df (pd.DataFrame): Dataframe to search for zeros and nans.
        cols (list, optional): List of columns, if no columns are specified all will be used. Defaults to None.
        thresholds (dict, optional): Thresholds for further . Defaults to None.

    Returns:
        pd.DataFrame: Dataframe containing counts of zeros and nans.
    """
    # use all columns af none were defined
    if cols == None:
        cols=df.columns
    # make a new dataframe and put the defined column names in the first column
    df_nan_0 = pd.DataFrame()
    df_nan_0['data'] = cols
    # calculate missing values and zeros as absolute value and share 
    df_nan_0['missing_values'] = [df[col].isna().sum() for col in cols]
    df_nan_0['missing_values_share'] = [df[col].isna().sum() / df.shape[0] * 100 for col in cols]
    df_nan_0['0_values'] = [df[df[col]==0][col].count() for col in cols]
    df_nan_0['0_values_share'] = [df[df[col]==0][col].count() / df.shape[0] * 100 for col in cols]

    # transpose the dataframe and use the original column names as column names
    df_nan_0 = df_nan_0.set_index('data').T.reset_index()
    df_nan_0.columns = [name if i>0 else 'metric' for i, name in enumerate(df_nan_0.columns)]
    return df_nan_0


# find missing values and zeros in the sc dataset
df_data_analysis = count_nan_and_0s(df, data_cols)
df_data_analysis.round(1)

In [ ]:
# define metrics and columns to plot
metrics = ["missing_values_share", "0_values_share"]
ys = list(df_data_analysis.columns)
ys.remove('metric')

# define size of subplot
columns = 4
rows = int(np.ceil((len(df_data_analysis.columns) - 1) / columns))

# plot
fig, ax = plt.subplots(rows, columns, figsize=(20,20)) # create subplots
plt.suptitle("Data analysis of missing values and zeros", fontsize=20) # title of plot
fig.tight_layout() # tight_layout automatically adjusts subplot params so that the subplot(s) fits in to the figure area
plt.subplots_adjust(hspace = .5, wspace = .2, top = .93) # adjusts the space between the single subplots

for row in range(rows):
    for col in range(columns):
        if col + row * columns < len(ys):
        
            # create a bar for each metric defined above for a column of ys list
            sns.barplot(data=df_data_analysis[df_data_analysis['metric'].isin(metrics)], x='metric', y=ys[col + row * columns], ax=ax[row][col])
            # set ylim to [0, 100] as we are plotting percentages
            ax[row][col].set_ylim([0, 100])
            # put the percentage above each plotted bar
            ax[row][col].bar_label(ax[row][col].containers[0], fmt='%.1f')
            # set the x, y and x-tick labels
            ax[row][col].set_xlabel("")
            ax[row][col].set_ylabel("Share of values in %")
            ax[row][col].set_xticklabels(labels=["Missing values", "Zeros"])
            # use the column name with slight changes as subplot name
            title = f"{ys[col + row * columns]}".replace('_', ' ').replace('std', 'std. dev.').replace('2p5', '2.5').capitalize()
            ax[row][col].set_title(title, fontsize = 15);
        else:
            # delete not needed subplots
            fig.delaxes(ax[row][col])


In [ ]:
# columns to plot
ys = data_cols_wo_std

# define size of subplot
columns = 3
rows = int(np.ceil((len(ys)) / columns))

# plot
fig, ax = plt.subplots(rows, columns, figsize=(20,20)) # create subplots
plt.suptitle("Outlier analysis", fontsize=20) # title of plot
fig.tight_layout() # tight_layout automatically adjusts subplot params so that the subplot(s) fits in to the figure area
plt.subplots_adjust(hspace = .7, wspace = .2, top = .93) # adjusts the space between the single subplots

for row in range(rows):
    for col in range(columns):
        if col + row * columns < len(ys):

            # create a bar for each metric defined above for a column of ys list
            sns.scatterplot(data=df, x='timestamp', y=ys[col + row * columns], ax=ax[row][col], alpha=.3)
            # set the x, y and x-tick labels
            ax[row][col].set_xlabel(ax[row][col].get_xlabel().capitalize())
            ax[row][col].set_ylabel(ax[row][col].get_ylabel().capitalize())
            # use the column name with slight changes as subplot name
            title = f"{ys[col + row * columns]}".replace('_', ' ').replace('std', 'std. dev.').replace('2p5', '2.5').capitalize()
            ax[row][col].set_title(title, fontsize = 15)
            ax[row][col].tick_params(labelrotation=90)
        else:
            # delete not needed subplots
            fig.delaxes(ax[row][col])


There are few outliers in humidity, pressure and temperature which can be dropped by setting thresholds. </br>
For PM10 and PM2.5 it is less obvious as the data is scattered all over the possible range. 

# Delete unrealistic values and outliers for environmental variables

## hard thresholds based on physical estimations
We can first have a look at the extreme values measured by Deutscher Wetterdienst to get an impression what range of values is realistic.

In [ ]:
print(df['humidity_dwd'].max())
print(df.query("city == 'Frankfurt'")['humidity_dwd'].min())
print(df.query("city == 'Bremen'")['humidity_dwd'].min())

In [ ]:
print(df['pressure_dwd'].max())
print(df.query("city == 'Frankfurt'")['pressure_dwd'].min())
print(df.query("city == 'Bremen'")['pressure_dwd'].min())

In [ ]:
print(df['temperature_dwd'].max())
print(df.query("city == 'Frankfurt'")['temperature_dwd'].min())
print(df.query("city == 'Bremen'")['temperature_dwd'].min())

In [ ]:
# set lower and upper threshold
thresholds_env = {
    'humidity_sensors': (15, 100),
    'pressure_sensors': (960, 1050),
    'temperature_sensors': (-20, 60),
}

def del_hard_thresholds_env(df, thresholds_env=thresholds_env):
    # delete values below lower and above upper threshold
    for col, thresh in thresholds_env.items():
        nan_before = df[col].isna().sum()
        df.iloc[df[col] <= thresh[0], list(df.columns).index(col)] = np.nan
        df.iloc[df[col] >= thresh[1], list(df.columns).index(col)] = np.nan
        print(f"added {df[col].isna().sum() - nan_before} nans in {col}")

print(df['temperature_sensors'].isna().sum())
del_hard_thresholds_env(df)
print(df['temperature_sensors'].isna().sum())


## values with std. dev. 'nan' or zero
If the standard deviation is 'nan', there was no or only one observation. If the standard deviation is zero, there was no fluctuation in the measured value, what can be assumed to be a measurement error.

In [ ]:
# delete values for the defined columns if the standard deviation is zero or 'nan'
cols_env = [
    'temperature_sensors',
    'humidity_sensors',
    'pressure_sensors',
]

def del_std_nan_env(df, cols=cols_env):
    for col in cols:
        nan_before = df[col].isna().sum()
        df.loc[df[col.split('_')[0]+'_std']==0, col] = np.nan    
        df.loc[df[col.split('_')[0]+'_std']==np.nan, col] = np.nan    
        print(f"added {df[col].isna().sum() - nan_before} nans in {col}")

print(df['temperature_sensors'].isna().sum())
del_std_nan_env(df)
print(df['temperature_sensors'].isna().sum())


## dynamic thresholds based on quantiles

In [ ]:
# define quantiles as threshold
thresh = {
    'temperature': (.01, .85),
    'humidity': (.05, .95),
    'pressure': (.05, .95),
}


def del_dynamic_threshold_env(df, thresh=thresh):
# make a dataframe containing median, upper and lower threshold defined by the quantiles above
    df_thresholds = df.groupby(['city', 'timestamp']).agg(
        temp_median = pd.NamedAgg(column='temperature_sensors', aggfunc='median'), 
        temp_lower = pd.NamedAgg(column='temperature_sensors', aggfunc=lambda x: x.quantile(q=thresh['temperature'][0])),
        temp_upper = pd.NamedAgg(column='temperature_sensors', aggfunc=lambda x: x.quantile(q=thresh['temperature'][1])),
        hum_median = pd.NamedAgg(column='humidity_sensors', aggfunc='median'), 
        hum_lower = pd.NamedAgg(column='humidity_sensors', aggfunc=lambda x: x.quantile(q=thresh['humidity'][0])),
        hum_upper = pd.NamedAgg(column='humidity_sensors', aggfunc=lambda x: x.quantile(q=thresh['humidity'][1])),
        pres_median = pd.NamedAgg(column='pressure_sensors', aggfunc='median'), 
        pres_lower = pd.NamedAgg(column='pressure_sensors', aggfunc=lambda x: x.quantile(q=thresh['pressure'][0])),
        pres_upper = pd.NamedAgg(column='pressure_sensors', aggfunc=lambda x: x.quantile(q=thresh['pressure'][1])),
    ).reset_index()

    # merge the thresholds with the sc dataframe
    df = df.merge(df_thresholds, how='left', on=['city', 'timestamp'])

    # replace values below lower threshold and above upper threshold with 'nan'
    for col, thresholds in {
        'temperature_sensors': ['temp_lower', 'temp_upper'],
        'humidity_sensors': ['hum_lower', 'hum_upper'],
        'pressure_sensors': ['pres_lower','pres_upper'],
    }.items():
        nan_before = df[col].isna().sum()
        df.loc[(df[col] < df[thresholds[0]]) | (df[col] > df[thresholds[1]]), col] = np.nan
        print(f"{df[col].isna().sum() - nan_before} nans added in {col}")

    # drop columns used for dynamic thresholding
    df.drop([col for col in df_thresholds.columns if not col in no_data_cols], axis=1, inplace=True)

print(df['temperature_sensors'].isna().sum())
del_dynamic_threshold_env(df)
print(df['temperature_sensors'].isna().sum())

In [ ]:
# df_thresholds.columns

In [ ]:
# # replace values below lower threshold and above upper threshold with 'nan'
# for col, thresholds in {
#     'temperature_sensors': ['temp_lower', 'temp_upper'],
#     'humidity_sensors': ['hum_lower', 'hum_upper'],
#     'pressure_sensors': ['pres_lower','pres_upper'],
# }.items():
#     nan_before = df[col].isna().sum()
#     df.loc[(df[col] < df[thresholds[0]]) | (df[col] > df[thresholds[1]]), col] = np.nan
#     print(f"{df[col].isna().sum() - nan_before} nans added in {col}")


In [ ]:
# # drop columns used for dynamic thresholding
# df.drop([col for col in df_thresholds.columns if not col in no_data_cols], axis=1, inplace=True)

# Visualization of cleaned data and comparison with dwd data

In [ ]:
def plot_sc_vs_dwd(city, columns=1, reduction=1):
    # Plot dwd and sc data 
    # define size of subplot
    rows = int(np.ceil(3 / columns))

    fig, ax = plt.subplots(rows, columns, figsize=(20,20)) # create subplots
    plt.suptitle(f"Comparison sensor data vs. dwd in {city}", fontsize=20) # title of plot
    fig.tight_layout() # tight_layout automatically adjusts subplot params so that the subplot(s) fits in to the figure area
    plt.subplots_adjust(hspace = .2, wspace = .2, top = .95) # adjusts the space between the single subplots

    # Plot humidity from both datasets vs time
    sns.scatterplot(data=df[(df['humidity_sensors'].notna()) & (df['city'] == city)][::reduction], x='timestamp', y='humidity_sensors', ax=ax[0], label='Sensor Community')
    sns.lineplot(data=df[(df['humidity_dwd'].notna()) & (df['city']== city)], x='timestamp', y='humidity_dwd', color='red', alpha=.5, ax=ax[0], label='Deutscher Wetterdienst')
    ax[0].set_ylabel('Relative Humidity in %')

    # Plot pressure from both datasets vs time
    sns.scatterplot(data=df[(df['pressure_sensors'].notna()) & (df['city'] == city)][::reduction], x='timestamp', y='pressure_sensors', ax=ax[1], label='Sensor Community')
    sns.lineplot(data=df[(df['pressure_dwd'].notna()) & (df['city']== city)], x='timestamp', y='pressure_dwd', color='red', alpha=.5, ax=ax[1], label='Deutscher Wetterdienst')
    ax[1].set_ylabel('Pressure in hPa')

    # Plot temperature from both datasets vs time
    sns.scatterplot(data=df[(df['temperature_sensors'].notna()) & (df['city'] == city)][::reduction], x='timestamp', y='temperature_sensors', ax=ax[2], label='Sensor Community')
    sns.lineplot(data=df[(df['temperature_dwd'].notna()) & (df['city']== city)], x='timestamp', y='temperature_dwd', color='red', alpha=.5, ax=ax[2], label='Deutscher Wetterdienst')
    ax[2].set_ylabel('Temperature in °C')

    xlim_left = df['timestamp'].min()
    xlim_right = df['timestamp'].max()

    # capitalize axis titles and add legend
    for i in range(3):
        ax[i].legend(loc='lower right')
        ax[i].set_xlabel(ax[i].get_xlabel().capitalize())
        ax[i].set_xlim(xlim_left, xlim_right)
    
    


In [ ]:
# # Plot comparison of data from both sources for Frankfurt
# plot_sc_vs_dwd('Frankfurt')
# plt.savefig("../figures/EDA_sc_vs_dwd_Frankfurt.png", bbox_inches='tight')
# plt.close()
# ;

![EDA_sc_vs_dwd_Frankfurt.png](../figures/EDA_sc_vs_dwd_Frankfurt.png)


In [ ]:
# # Plot comparison of data from both sources for Bremen
# plot_sc_vs_dwd('Bremen')
# plt.savefig("../figures/EDA_sc_vs_dwd_Bremen.png", bbox_inches='tight')
# plt.close()
# ;

![EDA_sc_vs_dwd_Bremen.png](../figures/EDA_sc_vs_dwd_Bremen.png)


In [ ]:
# Example of the distribution of measured temperatures in one day
sns.histplot(data=df[(df['timestamp'] > '2020-07-01') & (df['timestamp'] < '2020-07-15')], x='temperature_sensors', bins=20);

# Investigation of single locations

In [ ]:
# group by location_id and calculate the total number of hours with measurements, date of the first and of the last measurement
location_grouped = df[(df['PM10'].notna()) & (df['PM2p5'].notna())][['location_id', 'timestamp']].\
    groupby(['location_id']).\
        agg(
                hours = pd.NamedAgg(column='timestamp', aggfunc='count'), 
                date_min = pd.NamedAgg(column='timestamp', aggfunc='min'),
                date_max = pd.NamedAgg(column='timestamp', aggfunc='max')
            ).\
            reset_index().\
                sort_values('hours', ascending=False)

location_grouped['date_min'] = pd.to_datetime(location_grouped['date_min'])
location_grouped['date_max'] = pd.to_datetime(location_grouped['date_max'])
location_grouped['period_length'] = location_grouped['date_max'] - location_grouped['date_min'] + pd.Timedelta(days=1)
location_grouped['hours_per_day'] = location_grouped['hours'] / location_grouped['period_length'].dt.days
location_grouped.head(5)

In [ ]:
# plot the number of hours that were measured at each location
plt.figure(figsize=(25, 10))
g = sns.barplot(data=location_grouped, x='location_id', y='hours', order=location_grouped.sort_values('hours', ascending=False)['location_id'])
g.set_xlabel(g.get_xlabel().capitalize().replace('_', ' '))
g.set_ylabel(g.get_ylabel().capitalize())
plt.xticks(rotation=90);

In [ ]:
# plot the number of hours per day measured per location
plt.figure(figsize=(25, 10))
g = sns.barplot(data=location_grouped.sort_values('hours_per_day', ascending=False), x='location_id', y='hours_per_day', order=location_grouped.sort_values('hours_per_day', ascending=False)['location_id'])
g.set_xlabel(g.get_xlabel().capitalize().replace('_', ' '))
g.set_ylabel(g.get_ylabel().capitalize().replace('_', ' '))
plt.xticks(rotation=90);

In [ ]:
print(f"Total number of locations: {location_grouped.shape[0]}")
print('Locations with the least hours of measurement:')
location_grouped.tail(20)

In [ ]:
location_grouped[['hours', 'hours_per_day']].describe().T.round(1)

There are some sensor locations which delivered data only for few hours

In [ ]:
def plot_all_PM(df):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(20, 20))
    plt.suptitle("Sensors per City", fontsize=20) # title of plot
    fig.tight_layout() # tight_layout automatically adjusts subplot params so that the subplot(s) fits in to the figure area
    plt.subplots_adjust(hspace = .5, wspace = .2, top = .9) # adjusts the space between the single subplots

    # get ids match them with the cities
    labels_frankfurt = set(df.query("city=='Frankfurt'")['location_id'])
    labels_bremen = set(df.query("city=='Bremen'")['location_id'])

    # plot PM10 data of Frankfurt
    sns.lineplot(data=df[df['city']=='Frankfurt'][::10], x='timestamp', y='PM10', hue='location_id', ax=ax1, legend=False)
    ax1.legend(labels=labels_frankfurt) # assign a unique color to every id
    ax1.set_title('Frankfurt - PM10', fontsize = 15) # set title and font size
    ax1.legend([], [], frameon=False) # hide legend

    # plot PM2.5 data for Frankfurt
    sns.lineplot(data=df[df['city']=='Frankfurt'][::10], x='timestamp', y='PM2p5', hue='location_id', ax=ax2, legend=False)
    ax2.legend(labels=labels_frankfurt)
    ax2.set_title('Frankfurt - PM2.5', fontsize = 15)
    ax2.legend([], [], frameon=False)

    # plot PM10 data for Bremen
    sns.lineplot(data=df[df['city']=='Bremen'][::10], x='timestamp', y='PM10', hue='location_id', ax=ax3, legend=False)
    ax3.legend(labels=labels_bremen)
    ax3.set_title('Bremen - PM10', fontsize = 15)
    ax3.legend([], [], frameon=False)

    # plot PM2.5 data for Bremen
    sns.lineplot(data=df[df['city']=='Bremen'][::10], x='timestamp', y='PM2p5', hue='location_id', ax=ax4, legend=False)
    ax4.legend(labels=labels_bremen)
    ax4.set_title('Bremen - PM2.5', fontsize = 15)
    ax4.legend([], [], frameon=False)

plot_all_PM(df)

# Example location (location_id=2)

In [ ]:
# get location_id's occuring in Frankfurt
ids_frankfurt = df.query("city=='Frankfurt'")['location_id'].unique()

# plot PM10, PM2.5 and humidity of one location
plt.figure(figsize=(15, 8))
ax = sns.lineplot(data=df[df['location_id']==ids_frankfurt[0]], x='timestamp', y='PM10', color='b', alpha=.5)
sns.lineplot(data=df[df['location_id']==ids_frankfurt[0]], x='timestamp', y='PM2p5', color="r", alpha=.5, ax=ax)
ax2 = ax.twinx() # add second y-axis
sns.lineplot(data=df[df['location_id']==ids_frankfurt[0]], x='timestamp', y='humidity_sensors', color="g", alpha=.5, ax=ax2)



In [ ]:
# Plot correlation heatmap for one single location
sns.heatmap(df[df['location_id']==ids_frankfurt[0]][sc_cols_wo_std].corr(), annot=True)

In [ ]:
# make a dataframe containing timestamps of one year with resolution of one hour
one_year_full = pd.DataFrame()
one_year_full['timestamp'] = pd.date_range("2021-03-01", "2022-02-28 23:00:00", freq="H")

In [ ]:
# add observations of one location to that dataframe
one_year_full_2 = pd.merge(one_year_full, df[df['location_id']==ids_frankfurt[0]], how='left', on='timestamp')
print(f"{one_year_full_2['PM10'].isna().sum()} missing values in PM10")
print(f"{one_year_full_2['PM2p5'].isna().sum()} missing values in PM2.5")

In [ ]:
# get indices of observations where PM10 value is 'NaN'
missing_index = one_year_full_2.index[one_year_full_2['PM10'].isna()].tolist()

missing_periods = [] # list for periods of missing values
i = 0 # index for loop
start = None # start of a period
previous = None # index of the previous loop


while i < len(missing_index):
    # if start is None, it is the first loop
    if start == None:
        start = previous = missing_index[i]
        i += 1
        continue
    # if the current index is the previous index + 1, we are still moving within a closed period
    if missing_index[i] == previous+1:
        previous = missing_index[i]
        i += 1
        continue
    # else one period is over and another one is starting
    else:
        # add the closed period to the list of missing periods
        missing_periods.append(
            (one_year_full_2['timestamp'][start], 
            one_year_full_2['timestamp'][previous], 
            one_year_full_2['timestamp'][previous] - one_year_full_2['timestamp'][start] + pd.Timedelta(1, 'hour'))
        )
        start = previous = missing_index[i]
        i += 1
# add the last period to the list
missing_periods.append(
    (one_year_full_2['timestamp'][start], 
    one_year_full_2['timestamp'][previous], 
    one_year_full_2['timestamp'][previous]  - one_year_full_2['timestamp'][start] + pd.Timedelta(1, 'hour'))
)

# print the periods of missing PM10 values and their duration
p = 0
for start, end, duration in missing_periods:
    p += 1
    print(f"Period of missing values #{p}:\n\tstart: {start}\n\tend: {end}\n\tduration: {duration}\n")

## Set dynamic thresholds for PM data

GOAL: Calculate a dynamic median per hour for all sensors in a city. If a value is for example three times the median it is estimated to be an error.

In [ ]:
def clean_pm(df: pd.DataFrame, cols: list=['PM10', 'PM2p5'], factor: int = 3) -> pd.DataFrame:
    """deletes outliers for the given columns and considerung their timestamps and cities which are larger than factor times the median

    Args:
        df (pd.DataFrame): input dataframe
        cols (list): columns to clean
        factor (int, optional): factor that is used to calculate the threshold for keeping or deleting data. Defaults to 3.

    Returns:
        pd.DataFrame: cleaned dataframe
    """

    for col in df.columns:
        if 'threshold' in col:
            df.drop(col, axis=1, inplace=True)
    
    # define a list for saving the thresholds
    thresholds = []

    # for each city in the dataframe make a dataframe with timestamps
    for city in df['city'].unique():
        df_cur = df[df['city'] == city]
        df_threshold = pd.DataFrame(
            data={
                'timestamp': df_cur['timestamp'].unique(), 
                'city': city
            }
        )

        # for each timestamp calculate the median and threshold (factor * median)
        for col in cols:
            df_threshold[col+'_median'] = df_threshold.apply(lambda x: df_cur[(df_cur['timestamp'] == x['timestamp'])][col].median(), axis=1)
            df_threshold[col+'_threshold'] = factor * df_threshold[col+'_median']
        thresholds.append(df_threshold)

    # concatenate all thresholds
    df_thresholds = pd.DataFrame()
    for df_threshold in thresholds:
        df_thresholds = pd.concat([df_thresholds, df_threshold])
    
    # merge thresholds with original dataframe on timestamp and city 
    df = df.merge(df_thresholds, how='left', on=['timestamp', 'city'])
    
    # delete values if they are above the threshold and print number of deleted values
    for col in cols:
        nan_before = df[col].isna().sum()
        df[col] = df.apply(lambda x: x[col] if x[col] <= x[col+'_threshold'] else np.nan, axis=1)
        print(f"{df[col].isna().sum() - nan_before} NaNs added in {col}")

    # for col in cols:
    #     df.drop([col+'_threshold'], axis=1, inplace=True)
    return df


df = clean_pm(df)


In [ ]:
plot_all_PM(df)

In [ ]:
def get_PM_data_per_location(df: pd.DataFrame) -> tuple:
    """

    Args:
        df (pd.Dataframe): Dataframe containing data of PM sensors

    Returns:
        tuple: Tuple containing one dataframe per city and PM sensor
    """
    # make dataframe containing the timestamps
    df_missing_values_bremen_pm10 = pd.DataFrame(
        data={
            'timestamp': df['timestamp'].unique(),
        }
    )

    # copy that dataframe for every combination of PM sensor and city
    df_missing_values_bremen_pm2p5 = df_missing_values_bremen_pm10.copy()
    df_missing_values_frankfurt_pm10 = df_missing_values_bremen_pm10.copy()
    df_missing_values_frankfurt_pm2p5 = df_missing_values_bremen_pm10.copy()

    # add sensor data for every location in Bremen
    for location in df.loc[df['city'] == 'Bremen', 'location_id'].unique():
        df_missing_values_bremen_pm10 = pd.merge(df_missing_values_bremen_pm10, df.loc[df['location_id']==location, ['timestamp','PM10']], on='timestamp')
        df_missing_values_bremen_pm10.rename(columns={'PM10': location}, inplace=True) # rename the new column using the location_id
        df_missing_values_bremen_pm10.set_index('timestamp', inplace=True) # use timestamps as index

        df_missing_values_bremen_pm2p5 = pd.merge(df_missing_values_bremen_pm2p5, df.loc[df['location_id']==location, ['timestamp','PM2p5']], on='timestamp')
        df_missing_values_bremen_pm2p5.rename(columns={'PM2p5': location}, inplace=True)
        df_missing_values_bremen_pm2p5.set_index('timestamp', inplace=True)

    # do the same for Frankfurt
    for location in df.loc[df['city'] == 'Frankfurt', 'location_id'].unique():
        df_missing_values_frankfurt_pm10 = pd.merge(df_missing_values_frankfurt_pm10, df.loc[df['location_id']==location, ['timestamp','PM10']], on='timestamp')
        df_missing_values_frankfurt_pm10.rename(columns={'PM10': location}, inplace=True)
        df_missing_values_frankfurt_pm10.set_index('timestamp', inplace=True)

        df_missing_values_frankfurt_pm2p5 = pd.merge(df_missing_values_frankfurt_pm2p5, df.loc[df['location_id']==location, ['timestamp','PM2p5']], on='timestamp')
        df_missing_values_frankfurt_pm2p5.rename(columns={'PM2p5': location}, inplace=True)
        df_missing_values_frankfurt_pm2p5.set_index('timestamp', inplace=True)
    return  df_missing_values_bremen_pm10, df_missing_values_bremen_pm2p5, df_missing_values_frankfurt_pm10, df_missing_values_frankfurt_pm2p5


df_missing_values_bremen_pm10, df_missing_values_bremen_pm2p5, df_missing_values_frankfurt_pm10, df_missing_values_frankfurt_pm2p5 = get_PM_data_per_location(df)


In [ ]:
# plot missing values per id for PM10 in Bremen
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_bremen_pm10.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM10 - Bremen', fontsize=20);

In [ ]:
# plot missing values per id for PM2.5 in Bremen
plt.figure(figsize=(30, 15))
g = sns.heatmap(df_missing_values_bremen_pm2p5.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM2.5 - Bremen', fontsize=20);

In [ ]:
# plot missing values per id for PM2.5 in Frankfurt
plt.figure(figsize=(30, 15))
g = sns.heatmap(df_missing_values_frankfurt_pm2p5.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM2.5 - Frankfurt', fontsize=20);

In [ ]:
# plot missing values per id for PM10 in Frankfurt
plt.figure(figsize=(30, 15))
g = sns.heatmap(df_missing_values_frankfurt_pm10.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM10 - Frankfurt', fontsize=20);

# Drop sensors with only few data in the past year

In [ ]:
# df.to_csv("../data/df_backup.csv")

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# import warnings
# warnings.filterwarnings("ignore")

# sns.set_theme()
# plt.rcParams.update({'figure.facecolor':'white'})

# df = pd.read_csv("../data/df_backup.csv", index_col=0)
# df['timestamp'] = pd.to_datetime(df['timestamp'])


In [ ]:
def get_share_of_missing_values(df: pd.DataFrame, start_time: str):
    # Get the total number of observations possible in the past year
    observations_of_interest = df[(df['location_id'] == df['location_id'].unique()[0]) & (df['timestamp'] >= pd.to_datetime(start_time))].shape[0]

    # make a dataframe to store missing values per location
    missing_values = pd.DataFrame(columns=['location_id', 'city', 'PM10_missing', 'PM2p5_missing'])

    # get missing values for every location
    for location in df['location_id'].unique():
        # filter for location
        df_cur = df[(df['location_id'] == location) & (df['timestamp'] >= pd.to_datetime('2021-01-01'))][['city', 'PM10', 'PM2p5']]
        
        # create a new entry in the dataframe containing location_id, city and share of missing values
        new_entry = {
            'location_id': int(location),
            'city': df_cur['city'].iloc[0],
            'PM10_missing': df_cur['PM10'].isna().sum() / observations_of_interest,
            'PM2p5_missing': df_cur['PM2p5'].isna().sum() / observations_of_interest,
        }
        missing_values = missing_values.append(new_entry, ignore_index=True)

    # cast location_id to int
    missing_values['location_id'] = missing_values['location_id'].astype(int) 
    return missing_values

In [ ]:

missing_values = get_share_of_missing_values(df, '2021-01-01')
missing_values

In [ ]:
fig, ax = plt.subplots(4,1,figsize=(20,15))
plt.suptitle("Missing values per city and sensor", fontsize=20) # title of plot
fig.tight_layout() # tight_layout automatically adjusts subplot params so that the subplot(s) fits in to the figure area
plt.subplots_adjust(hspace = .4, wspace = .2, top = .93) # adjusts the space between the single subplots
i=0
# plot share of missing values for every city and PM sensor
for city in missing_values['city'].unique():
    for col in ['PM10_missing', 'PM2p5_missing']:
        sns.barplot(
            data=missing_values[missing_values['city']==city],
            x='location_id',
            y=col,
            order=missing_values[missing_values['city']==city].sort_values(col, ascending=False)['location_id'], # sort by missing values
            ax=ax[i]
        )
        ax[i].tick_params(labelrotation=90) # rotate x tick labels
        ax[i].set_title(city + ' - ' + col.split('_')[0].replace('p', '.')) # set a title (City - Sensor)
        i += 1


In [ ]:
# get the IDs of good sensors having less than 25 % missing values in PM2.5
good_sensors = missing_values.query("PM2p5_missing < 0.25")['location_id']
good_sensors

In [ ]:
# get the data of those good sensors
def use_good_sensors_only(df, good_sensors=good_sensors):
    df_good_sensors = df[df['location_id'].\
        isin(good_sensors)].\
            drop([col for col in df.columns if ('median' in col or 'threshold' in col)], axis=1)
    return df_good_sensors

df_good_sensors = use_good_sensors_only(df)

In [ ]:
print(df.shape)
print(df_good_sensors.shape)

# Clean test data

In [ ]:
# import test data
df_test = pd.read_csv("../data/processed_sensor_dwd_test.csv", index_col=0)


In [ ]:
# assign location IDs according to coordinates
df_test['location_id'] = df_test.apply(
    lambda x: location_id_assignment.\
        loc[(location_id_assignment['lat'] == x['lat']) & (location_id_assignment['lon'] == x['lon']), 'location_id'].\
            iloc[0], 
            axis=1
)
df_test['location_id'].isna().sum()

In [ ]:
# convert timestamp to datetime
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

# sort columns
df_test = df_test.reindex(columns=no_data_cols + sc_cols + dwd_cols)

df_test.head()

In [ ]:
# remove outliers of environmental parameters by different mechanisms
print("hard thresholds")
del_hard_thresholds_env(df_test)

print("constant values")
del_std_nan_env(df_test)

print("dnyamic thersholds")
del_dynamic_threshold_env(df_test)

In [ ]:
# plot all PM data
plot_all_PM(df_test)

In [ ]:
# get missing values of PM data per sensor
df_missing_values_bremen_pm10_test, \
df_missing_values_bremen_pm2p5_test, \
df_missing_values_frankfurt_pm10_test, \
df_missing_values_frankfurt_pm2p5_test = get_PM_data_per_location(df_test)

In [ ]:
# plot missing values per id for PM10 in Bremen
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_bremen_pm10_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM10 - Bremen', fontsize=20);

In [ ]:
# plot missing values per id for PM2.5 in Bremen
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_bremen_pm2p5_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM2.5 - Bremen', fontsize=20);

In [ ]:
# plot missing values per id for PM10 in Frankfurt
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_frankfurt_pm10_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM10 - Frankfurt', fontsize=20);

In [ ]:
# plot missing values per id for PM2.5 in Frankfurt
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_frankfurt_pm2p5_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM2.5 - Frankfurt', fontsize=20);

In [ ]:
# clean PM data
df_test = clean_pm(df_test)


In [ ]:
# get data of sensors marked as good
df_good_sensors_test = use_good_sensors_only(df_test)

In [ ]:
print(df_test.shape)
print(df_good_sensors_test.shape)

## Test data after cleaning

In [ ]:
# number of 'good sensors' should be identical to locations in test dataframe
print(len(good_sensors))
df_good_sensors_test['location_id'].nunique()

In [ ]:
# plot all PM data per location
plot_all_PM(df_good_sensors_test)

In [ ]:
# get missing values per sensor
df_missing_values_bremen_pm10_test, \
df_missing_values_bremen_pm2p5_test, \
df_missing_values_frankfurt_pm10_test, \
df_missing_values_frankfurt_pm2p5_test = get_PM_data_per_location(df_good_sensors_test)

In [ ]:
# plot missing values per id for PM10 in Bremen
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_bremen_pm10_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM10 - Bremen', fontsize=20);

In [ ]:
# plot missing values per id for PM2.5 in Bremen
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_bremen_pm2p5_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM2.5 - Bremen', fontsize=20);

In [ ]:
# plot missing values per id for PM10 in Frankfurt
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_frankfurt_pm10_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM10 - Frankfurt', fontsize=20);

In [ ]:
# plot missing values per id for PM2.5 in Frankfurt
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_frankfurt_pm2p5_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM2.5 - Frankfurt', fontsize=20);

In [ ]:
# get share of missing values in the cleaned test dataframe
missing_values_test = get_share_of_missing_values(df_good_sensors_test, "2021-01-01")

# make a series of good sensors in test data (less than 75 % missing in PM2.5)
good_sensors_test = missing_values_test.query("PM2p5_missing < 0.25")['location_id']


In [ ]:
# get bad sensors in test dataframe (more than 75 % of PM2.5 data missing)
bad_sensors = []
for location in list(good_sensors):
    if location not in list(good_sensors_test):
        bad_sensors.append(location)

print(len(bad_sensors))
bad_sensors

# Update dataframes using only 'good sensors' and save cleaned train and test dataframe

In [ ]:
# update train dataframe according to good sensors in test data
df_good_sensors = use_good_sensors_only(df, good_sensors_test)

# save train data for good sensors
df_good_sensors.to_csv("../data/cleaned_sensors_dwd_train.csv")

In [ ]:
# make test dataframe containing only good sensors
df_good_sensors_test = use_good_sensors_only(df_test, good_sensors_test)

# save test data
df_good_sensors_test.to_csv("../data/cleaned_sensors_dwd_test.csv")

# Last check of missing data in the final dataframes

In [ ]:
df_missing_values_bremen_pm10_test, \
df_missing_values_bremen_pm2p5_test, \
df_missing_values_frankfurt_pm10_test, \
df_missing_values_frankfurt_pm2p5_test = get_PM_data_per_location(df_good_sensors_test)

In [ ]:
# plot missing values per id for PM10 in Bremen
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_bremen_pm10_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM10 - Bremen - Cleaned Test Data', fontsize=20);

In [ ]:
# plot missing values per id for PM2.5 in Bremen
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_bremen_pm2p5_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM2.5 - Bremen - Cleaned Test Data', fontsize=20);

In [ ]:
# plot missing values per id for PM10 in Frankfurt
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_frankfurt_pm10_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM10 - Frankfurt - Cleaned Test Data', fontsize=20);

In [ ]:
# plot missing values per id for PM2.5 in Frankfurt
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_frankfurt_pm2p5_test.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM2.5 - Frankfurt - Cleaned Test Data', fontsize=20);

In [ ]:
df_missing_values_bremen_pm10, \
df_missing_values_bremen_pm2p5, \
df_missing_values_frankfurt_pm10, \
df_missing_values_frankfurt_pm2p5 = get_PM_data_per_location(df_good_sensors)

In [ ]:
# plot missing values per id for PM10 in Bremen
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_bremen_pm10.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM10 - Bremen - Cleaned Train Data', fontsize=20);

In [ ]:
# plot missing values per id for PM2.5 in Bremen
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_bremen_pm2p5.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM2.5 - Bremen - Cleaned Train Data', fontsize=20);

In [ ]:
# plot missing values per id for PM10 in Frankfurt
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_frankfurt_pm10.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM10 - Frankfurt - Cleaned Train Data', fontsize=20);

In [ ]:
# plot missing values per id for PM2.5 in Frankfurt
plt.figure(figsize=(30, 10))
g = sns.heatmap(df_missing_values_frankfurt_pm2p5.isna().T.sort_index(), cbar_kws={'label': 'Missing Data'})
g.set_title('PM2.5 - Frankfurt - Cleaned Train Data', fontsize=20);